In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from PIL import Image, ImageOps
from plotnine import (ggplot, aes, geom_map, geom_text, geom_label, 
                      ggtitle, element_blank, element_rect, 
                      scale_fill_manual, theme_minimal, theme) 
from pulp import (LpProblem, LpMinimize, LpVariable, lpSum, 
                  PULP_CBC_CMD, GLPK_CMD, LpStatus, value) 

In [2]:
df=pd.read_csv('census.csv')
df.head()

,county_id,county,population,COUNTYFP10,latitude,longitude
0,0,Adair,7496,1,41.328528,-94.478164
1,1,Adams,3704,3,41.021656,-94.696906
2,2,Allamakee,14061,5,43.274964,-91.382751
3,3,Appanoose,12317,7,40.744683,-92.870345
4,4,Audubon,5674,9,41.679178,-94.904312


In [3]:
df['COUNTYFP10']=df['COUNTYFP10'].astype(str).str.pad(3,fillchar='0')

In [4]:
#imports county shapefiles from MGGG
shapefile_iowa = gpd.read_file('IA_counties/IA_counties.shp')
shapefile_iowa.head()

,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,TOTPOP,...,TOTVOT12,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD,geometry
0,19,127,19127,Marshall,Marshall County,1482770678,1803086,+42.0416910,-092.9814523,40648,...,19064,10257,8472,335,17980,7652,9146,1182,1,"POLYGON ((-92.76679 42.12346, -92.76679 42.122..."
1,19,011,19011,Benton,Benton County,1855117342,5760770,+42.0925474,-092.0576300,26076,...,14023,6862,6940,221,13844,4678,8232,934,1,"POLYGON ((-91.94773 41.86186, -91.95514 41.861..."
2,19,041,19041,Clay,Clay County,1469139214,13866941,+43.0798220,-095.1497261,16667,...,8502,3385,4951,166,8617,2249,5877,491,4,"POLYGON ((-95.26926 43.25537, -95.26140 43.255..."
3,19,165,19165,Shelby,Shelby County,1530110414,1486135,+41.6790143,-095.3089173,12167,...,6483,2469,3911,103,6370,1662,4362,346,4,"POLYGON ((-95.20902 41.86371, -95.20890 41.863..."
4,19,043,19043,Clayton,Clayton County,2016405612,36586071,+42.8409979,-091.3235108,18129,...,9138,4806,4164,168,9129,3237,5317,575,1,"POLYGON ((-91.25080 42.64558, -91.25160 42.645..."


In [5]:
map_population_by_county_data = shapefile_iowa.merge(df, on='COUNTYFP10')
county_populations = np.array(df['population'])
state_population = sum(county_populations)
df.sort_values('population', ascending=False).head()

,county_id,county,population,COUNTYFP10,latitude,longitude
76,76,Polk,492401,153,41.684281,-93.569720
56,56,Linn,230299,113,42.077951,-91.597673
81,81,Scott,174669,163,41.641679,-90.622290
51,51,Johnson,152854,103,41.668737,-91.588812
6,6,Black Hawk,131144,013,42.472888,-92.306059


In [106]:
#create a list of permutations of 99 counties
#putting the * will convert the array into a list form
#np.random permutes 99 numbers
n_experiments=200
county_perms=[]
for i in range(n_experiments):
    county_perms.append([*np.random.permutation(99)])

In [ ]:
#keep it fully random and avoid the county_perms list
# for i in range(5):
#     df_dummy=df.sort_values(by='county_id', key=lambda col: np.random.permutation(99))
#     print(df_dummy.head())

In [107]:
for i in range(n_experiments):
    df_dummy=df.sort_values(by='county_id', key=lambda col: county_perms[i])
    print(df_dummy.head())

    county_id      county  population COUNTYFP10   latitude  longitude
2           2  Allamakee        14061        005  43.274964 -91.382751
38         38    Guthrie        10623        077  41.683573 -94.501272
55         55        Lee        33555        111  40.647588 -91.477157
76         76       Polk       492401        153  41.684281 -93.569720
26         26    Decatur         7645        053  40.736379 -93.784580
    county_id       county  population COUNTYFP10   latitude  longitude
95         95  Winneshiek        20070        191  43.292989 -91.850788
59         59        Lyon        11934        119  43.383580 -96.207201
25         25       Davis         9110        051  40.748089 -92.410345
97         97       Worth         7443        195  43.373491 -93.248533
27         27    Delaware        17488        055  42.473003 -91.359443
    county_id    county  population COUNTYFP10   latitude  longitude
66         66   Monona         8751        133  42.049432 -95.956566
32  